 # P4 : Anticipez les besoins en consommation électrique de bâtiments

In [1]:
import os
import wget
import pandas as pd
from ast import literal_eval
import plotly.express as px


In [2]:
write_data = True

# True : création d'un dossier Figures et Tableau
# dans lesquels seront créés les éléments qui serviront à la présentation
# et écriture des figures et tableaux dans ces dossier
#
# False : pas de création de dossier ni de figures ni de tableaux

if write_data is True:
    try:
        os.mkdir("./Figures/")
    except OSError as error:
        print(error)
    try:
        os.mkdir("./Tableaux/")
    except OSError as error:
        print(error)
else:
    print("""Visualisation uniquement dans le notebook
    pas de création de figures ni de tableaux""")


[Errno 17] File exists: './Figures/'
[Errno 17] File exists: './Tableaux/'


In [3]:
# ouverture du fichire csv
# utilise le fichier dans le répertoire si il existe
# sinon récupération avec l'url
BEB2015 = pd.read_csv("2015-building-energy-benchmarking.csv")


In [4]:
# ouverture du fichire csv
# utilise le fichier dans le répertoire si il existe
# sinon récupération avec l'url
BEB2016 = pd.read_csv("2016-building-energy-benchmarking.csv")


 Liste des colonnes présentes uniquement dans les données de 2015

In [5]:
# colonnes uniquement dans les données de 2015
BEB2015.columns.difference(BEB2016.columns)


Index(['2010 Census Tracts', 'City Council Districts', 'Comment',
       'GHGEmissions(MetricTonsCO2e)', 'GHGEmissionsIntensity(kgCO2e/ft2)',
       'Location', 'OtherFuelUse(kBtu)', 'SPD Beats',
       'Seattle Police Department Micro Community Policing Plan Areas',
       'Zip Codes'],
      dtype='object')

 Liste des colonnes présentes uniquement dans les données de 2016

In [6]:
# colonnes uniquement dans les données de 2016
BEB2016.columns.difference(BEB2015.columns)


Index(['Address', 'City', 'Comments', 'GHGEmissionsIntensity', 'Latitude',
       'Longitude', 'State', 'TotalGHGEmissions', 'ZipCode'],
      dtype='object')

 Les données de location en 2015 sont dans une seule colonne
 on va faire en sorte d'uniformiser avec les colonnes présentes en 2016

In [7]:
# extraction et ajout des données des dict (literal_eval)
# mise sous forme de serie (pd.Series)
# et suppression de la colonne d'origine
BEB2015 = pd.concat([
    BEB2015.drop(columns='Location'),
    BEB2015.Location.map(literal_eval).apply(pd.Series)
],
                    axis=1)
BEB2015 = pd.concat([
    BEB2015.drop(columns='human_address'),
    BEB2015.human_address.map(literal_eval).apply(pd.Series)
],
                    axis=1)


In [8]:
# renomages des colonnes pour correspondre aux données de 2016
BEB2015 = BEB2015.rename(
    columns={
        "latitude": "Latitude",
        "longitude": "Longitude",
        "address": "Address",
        "city": "City",
        "state": "State",
        "zip": "ZipCode"
    })


In [9]:
BEB2015.columns.difference(BEB2016.columns)

Index(['2010 Census Tracts', 'City Council Districts', 'Comment',
       'GHGEmissions(MetricTonsCO2e)', 'GHGEmissionsIntensity(kgCO2e/ft2)',
       'OtherFuelUse(kBtu)', 'SPD Beats',
       'Seattle Police Department Micro Community Policing Plan Areas',
       'Zip Codes'],
      dtype='object')

In [10]:
BEB2016.columns.difference(BEB2015.columns)

Index(['Comments', 'GHGEmissionsIntensity', 'TotalGHGEmissions'], dtype='object')

 GHGEMissions (MetricTonsCO2e) et TotalGHGEmissions renseignent les mêmes informations
 ainsi que GHGEmissionsIntesity (kgCO2e/ft2) et GHGEmissionsIntensity nous allons les
 renommer de la même manière (2016)

In [11]:
# renomages des colonnes pour correspondre aux données de 2016
BEB2015.rename(
    columns={
        "GHGEmissions(MetricTonsCO2e)": "TotalGHGEmissions",
        "GHGEmissionsIntensity(kgCO2e/ft2)": "GHGEmissionsIntensity",
    },
    inplace=True,
)


In [12]:
BEB2015.columns.difference(BEB2016.columns)

Index(['2010 Census Tracts', 'City Council Districts', 'Comment',
       'OtherFuelUse(kBtu)', 'SPD Beats',
       'Seattle Police Department Micro Community Policing Plan Areas',
       'Zip Codes'],
      dtype='object')

In [13]:
BEB2016.columns.difference(BEB2015.columns)

Index(['Comments'], dtype='object')

In [14]:
BEB2016.Comments.unique()


array([nan])

 Pas de commentaire dans les données de 2016

In [15]:
# sup. col. comments
BEB2016.drop(columns="Comments", inplace=True)

In [16]:
BEB2015.Comment.unique()

array([nan,
       'Under construction starting 6/2013 (old building demolished) and ending 9/2016. New Building re-opened 9/2016. Year built changed from 1948 to 2016.',
       "Part of McKinstry's campus, property operates a fabrication and production shop and is conditioned by natural gas overhead radiant unit heaters with no cooling. Learn more at www.mckinstry.com.",
       "One of two office buildings on McKinstry's campus partially remodeled in 2009 from warehouse to office space. Served by rooftop air handling units with underfloor VAVs.  Self-performed energy efficiency upgrades in 2015 include a long-term LED lighting re",
       'Part of McKinstry’s campus, this remodeled warehouse is mainly office but includes a full-size basketball court and gym with locker rooms, bistro with a full kitchen, and data center. Served by rooftop package units for heating and cooling. Upgrades inclu',
       'Construction completed in mid 2015. The building was unoccupied for most of 2015.',
 

 présence de commentaires dans les données de 2015

 Nous allons vérifier que les types des colonnes correspondent
 entre les deux jeux de données

In [17]:
# dataframe permettant de comparer les types des colonnes
# dans les deux jeux de donées
pd.DataFrame([BEB2015.dtypes, BEB2016.dtypes])


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,YearBuilt,NumberofBuildings,...,Seattle Police Department Micro Community Policing Plan Areas,City Council Districts,SPD Beats,Zip Codes,Latitude,Longitude,Address,City,State,ZipCode
0,int64,int64,object,object,object,object,int64,object,int64,int64,...,float64,float64,float64,int64,object,object,object,object,object,object
1,int64,int64,object,object,object,object,int64,object,int64,float64,...,NaN,NaN,NaN,NaN,float64,float64,object,object,object,float64


 Les colonnes latitude, longitude et zipcode de 2015
 ne sont pas reconnues comme des nombres nous allons y remédier

In [18]:
# lat, log et zip en décimaux
BEB2015[['Latitude', 'Longitude',
         'ZipCode']] = BEB2015[['Latitude', 'Longitude',
                                'ZipCode']].astype('float64')

 Nous allons joindre nos données pour n'avoir qu'un seul fichier
 sur lequel travailler lors des test de modèles

In [19]:
# jonction des deux jeux de données
BEBFull = BEB2015.merge(BEB2016, how="outer")

 Nous allons voir quelques statistiques sur chacunes de nos colonnes

In [20]:
# stats sur données catégorielles
StatsCat = BEBFull.describe(exclude='number')
StatsCat


,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,Neighborhood,ListOfAllPropertyUseTypes,LargestPropertyUseType,SecondLargestPropertyUseType,ThirdLargestPropertyUseType,YearsENERGYSTARCertified,DefaultData,Comment,ComplianceStatus,Outlier,Address,City,State
count,6716,6716,6716,6714,6716,6580,6560,3238,1156,229,6715,13,6716,116,6716,6716,6716
unique,8,32,6527,3729,19,488,57,50,45,116,4,13,5,4,6379,2,1
top,NonResidential,Low-Rise Multifamily,WAREHOUSE,3224049012,DOWNTOWN,Multifamily Housing,Multifamily Housing,Parking,Retail Store,2016,False,Under construction starting 6/2013 (old buildi...,Compliant,High Outlier,2623 NE UNIVERSITY VILLAGE ST,Seattle,WA
freq,2921,1985,9,10,1151,1728,3265,1857,217,34,3263,1,6548,46,8,3376,6716


In [21]:
# stats sur données numériques
StatsNum = BEBFull.describe()
StatsNum


,OSEBuildingID,DataYear,CouncilDistrictCode,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseTypeGFA,...,TotalGHGEmissions,GHGEmissionsIntensity,2010 Census Tracts,Seattle Police Department Micro Community Policing Plan Areas,City Council Districts,SPD Beats,Zip Codes,Latitude,Longitude,ZipCode
count,6716.000000,6716.00000,6716.000000,6716.000000,6708.000000,6708.000000,6.716000e+03,6716.000000,6.716000e+03,6.560000e+03,...,6697.000000,6697.000000,224.000000,3338.000000,213.000000,3338.000000,3340.000000,6716.000000,6716.000000,6700.000000
mean,20867.420488,2015.50268,4.445057,1968.101549,1.071407,4.695736,9.281410e+04,9519.597975,8.329450e+04,7.707936e+04,...,114.935638,1.081154,123.062500,32.380168,1.347418,24.825644,18786.680240,47.624062,-122.334904,98116.843582
std,11878.968174,0.50003,2.123749,33.020391,1.593505,5.476218,1.875062e+05,33868.050881,1.733470e+05,1.681617e+05,...,478.876573,1.734763,5.812128,19.907567,0.477272,15.006287,588.604192,0.047837,0.027185,17.730253
min,1.000000,2015.00000,1.000000,1900.000000,0.000000,0.000000,1.128500e+04,-3.000000,-5.055000e+04,5.656000e+03,...,-0.800000,-0.020000,116.000000,1.000000,1.000000,1.000000,17916.000000,47.499170,-122.414250,98006.000000
25%,19934.000000,2015.00000,3.000000,1948.000000,1.000000,2.000000,2.848700e+04,0.000000,2.727300e+04,2.501600e+04,...,9.450000,0.160000,117.000000,14.000000,1.000000,10.000000,18379.000000,47.599857,-122.350603,98105.000000
50%,23064.000000,2016.00000,4.000000,1974.000000,1.000000,4.000000,4.400800e+04,0.000000,4.218500e+04,3.953000e+04,...,33.180000,0.540000,123.000000,33.000000,1.000000,26.000000,18390.000000,47.618608,-122.332630,98115.000000
75%,25868.250000,2016.00000,7.000000,1996.000000,1.000000,5.000000,8.943125e+04,0.000000,8.112000e+04,7.417800e+04,...,91.530000,1.280000,125.500000,50.000000,2.000000,38.000000,19576.000000,47.657135,-122.319733,98122.000000
max,50226.000000,2016.00000,7.000000,2015.000000,111.000000,99.000000,9.320156e+06,512608.000000,9.320156e+06,9.320156e+06,...,16870.980000,34.090000,135.000000,61.000000,2.000000,51.000000,19584.000000,47.733870,-122.220470,98272.000000


 Nous avons des valeurs de surface de batiment et de parking négatives
 nous allons supprimer ces batiments

In [22]:
# sup. val < 0 dans les valeurs de surface
BEBFullClean = BEBFull.drop(BEBFull[(BEBFull['PropertyGFABuilding(s)'] < 0)
                                    | (BEBFull.PropertyGFAParking < 0)].index)


 Nous allons supprimer les colonnes comportant moins de 50% de données
 et celles qui ne nous intéressent pas
 (1 seule valeurs dans les colonnes state/city par exemple)

In [23]:
BEBFullClean.dropna(
    axis='columns',
    thresh=(BEBFullClean.shape[0] * .5),
    # nombre de valeurs = shape * .1
    # soit 90% de NaN et 10% de valeurs
    inplace=True)


In [24]:
# liste des valeurs dans la colonne City
BEBFullClean.City.unique()

array(['SEATTLE', 'Seattle'], dtype=object)

In [25]:
BEBFullClean.drop(columns=['State', 'City'], inplace=True)

In [26]:
# graphique du nombre de données par indicateurs après filtre NaN
px.bar(x=BEBFullClean.shape[0] -
       BEBFullClean.isna().sum().sort_values(ascending=False).values,
       y=BEBFullClean.isna().sum().sort_values(ascending=False).index,
       labels=dict(x='Nombre de données', y='Indicateurs'),
       height=1000,
       width=1000)

In [27]:
BEBFullClean.Neighborhood.unique()

array(['DOWNTOWN', 'SOUTHEAST', 'NORTHEAST', 'EAST', 'CENTRAL', 'NORTH',
       'MAGNOLIA / QUEEN ANNE', 'LAKE UNION', 'GREATER DUWAMISH',
       'BALLARD', 'NORTHWEST', 'SOUTHWEST', 'DELRIDGE', 'Central',
       'Ballard', 'North', 'Delridge', 'Northwest',
       'DELRIDGE NEIGHBORHOODS'], dtype=object)

In [28]:
BEBFullClean.Neighborhood.replace('DELRIDGE NEIGHBORHOODS',
                                  'DELRIDGE',
                                  inplace=True)
BEBFullClean.Neighborhood = BEBFullClean.Neighborhood.map(lambda x: x.upper())

In [29]:
BEBFullClean.Neighborhood.unique()

array(['DOWNTOWN', 'SOUTHEAST', 'NORTHEAST', 'EAST', 'CENTRAL', 'NORTH',
       'MAGNOLIA / QUEEN ANNE', 'LAKE UNION', 'GREATER DUWAMISH',
       'BALLARD', 'NORTHWEST', 'SOUTHWEST', 'DELRIDGE'], dtype=object)